In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics



# Train 데이터 전처리
train = pd.read_csv("../input/train_labels.csv")
train["path"] = train["name"].apply(lambda x: "../input/train/" + str(x) + ".jpg")
train["label"] = train["invasive"].astype("str")
train.drop(["name", "invasive"], 1, inplace=True)

val_size = 50
image_size = 500
batch_size = 18

train_gen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True, rotation_range=270, )
train_generator = train_gen.flow_from_dataframe(dataframe = train.iloc[:2245], x_col = "path", y_col = "label", 
                                                batch_size = batch_size, target_size = (image_size, image_size))

val_generator = train_gen.flow_from_dataframe(dataframe = train.iloc[2245:], x_col = "path", y_col = "label", 
                                                batch_size = batch_size, target_size = (image_size, image_size))

# 모델 구축 - InceptionResNetV2 활용
import keras
from keras.layers import Dense, Input, Flatten, Dropout, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.applications.inception_resnet_v2 import InceptionResNetV2 #InceptionRestNetV2 불러오기
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

modelcheck = ModelCheckpoint("best.h5", save_best_only=True)

#모델 층을 쌓는다 - Sequntial 방식
model = Sequential()
model.add(InceptionResNetV2(weights = "imagenet", pooling = "avg", input_shape = (image_size, image_size, 3), include_top = False))
model.add(Dropout(0.5)) #Drop out 통해 overfitting 방지
model.add(Dense(2, activation = "softmax")) # 출력층

#모델을 compile 
model.compile(loss = "categorical_crossentropy", optimizer = Adam(lr = 0.0003, decay = 2e-7))
#모델을 학습시킨다
model.fit_generator(train_generator, steps_per_epoch = int(np.ceil(2245/batch_size)), callbacks=[modelcheck],
                    validation_data=val_generator, validation_steps=int(np.ceil(50/batch_size)),
                   epochs = 10)


#test set 전처리
sub = pd.read_csv("../input/sample_submission.csv")
sub["path"] = sub["name"].apply(lambda x: "../input/test/" + str(x) + ".jpg")
sub.drop(["name", "invasive"],1,inplace=True)

test_gen = ImageDataGenerator(rescale = 1./255)
test_generator = test_gen.flow_from_dataframe(dataframe = sub, shuffle=False, x_col = "path", y_col = None, class_mode=None,
                                                batch_size = batch_size, target_size = (image_size, image_size))

model.load_weights("best.h5")

# 모델을 통해 예측한다
preds = model.predict_generator(test_generator, steps=int(np.ceil(1531/batch_size)), workers = 2)
sub = pd.read_csv("../input/sample_submission.csv")
sub["invasive"] = preds
sub.to_csv("inva2.csv", index=False)